# TP Final – InfoVis 2025 · COVID-19 Argentina

**Integrantes:** Leonardo Gabriel Vargas Kuhn  
**Fecha:** 09-09-2025  
**Fuente:** SNVS-SISA – datos.gob.ar  



In [6]:
# @title

YEARS_FILTER = [2020, 2021, 2022]
DATA_ZIP_URL = "https://sisa.msal.gov.ar/datos/descargas/covid-19/files/Covid19Casos.zip"

import pandas as pd, altair as alt, zipfile, urllib.request
from collections import defaultdict

alt.data_transformers.disable_max_rows()

# -------------------------
# Descargar ZIP
# -------------------------
zip_path = "/content/covid.zip"
urllib.request.urlretrieve(DATA_ZIP_URL, zip_path)

with zipfile.ZipFile(zip_path, "r") as z:
    csv_candidates = [n for n in z.namelist() if n.lower().endswith(".csv")]
    csv_name = csv_candidates[0]
    print("CSV detectado:", csv_name)

# -------------------------
# Procesar datos por chunks
# -------------------------
usecols = ["edad","sexo","residencia_provincia_nombre",
           "clasificacion_resumen","fallecido","fecha_diagnostico"]
chunksize = 500_000
agg_cases, agg_deaths = {}, {}

def add(d, k, v=1): d[k] = d.get(k, 0) + v

with zipfile.ZipFile(zip_path, "r") as z:
    with z.open(csv_name) as f:
        for chunk in pd.read_csv(f, sep=",", usecols=usecols, chunksize=chunksize,
                                 low_memory=False, dtype=str):
            chunk["fecha_diagnostico"] = pd.to_datetime(chunk["fecha_diagnostico"], errors="coerce")
            chunk = chunk.dropna(subset=["fecha_diagnostico"]).copy()
            chunk["year"] = chunk["fecha_diagnostico"].dt.year.astype(int)
            chunk = chunk[chunk["year"].isin(YEARS_FILTER)]
            chunk["clasificacion_resumen"] = chunk["clasificacion_resumen"].str.strip().str.lower()
            chunk = chunk[chunk["clasificacion_resumen"]=="confirmado"]
            chunk["provincia"] = chunk["residencia_provincia_nombre"].fillna("Sin dato")
            chunk["mes"] = chunk["fecha_diagnostico"].dt.to_period("M").dt.to_timestamp()

            for (m,p),cnt in chunk.groupby(["mes","provincia"]).size().items():
                add(agg_cases,(m,p),int(cnt))

            died = chunk[chunk["fallecido"].str.strip().str.upper()=="SI"]
            for (m,p),cnt in died.groupby(["mes","provincia"]).size().items():
                add(agg_deaths,(m,p),int(cnt))

df_cases = pd.DataFrame([{"mes":k[0],"provincia":k[1],"casos":v} for k,v in agg_cases.items()])
df_deaths = pd.DataFrame([{"mes":k[0],"provincia":k[1],"muertes":v} for k,v in agg_deaths.items()])
df = pd.merge(df_cases, df_deaths, on=["mes","provincia"], how="left").fillna(0)

print("Shape:", df.shape)
display(df.head())

# -------------------------
# EDA rápido
# -------------------------
print("Provincias:", df["provincia"].nunique())
print("Período:", str(df["mes"].min().date()), "→", str(df["mes"].max().date()))
top_prov = df.groupby("provincia", as_index=False)["casos"].sum().sort_values("casos", ascending=False).head(10)
display(top_prov)

# -------------------------
# VIS 1 – Serie nacional
# -------------------------
df_nat = df.groupby("mes", as_index=False)["casos"].sum()
alt.Chart(df_nat).mark_line(point=True).encode(
    x=alt.X("mes:T", title="Mes"),
    y=alt.Y("casos:Q", title="Casos confirmados")
).properties(
    title="Casos confirmados mensuales – Total país",
    width=700, height=350
)

# -------------------------
# VIS 2 – Top-6 provincias
# -------------------------
top6 = top_prov["provincia"].head(6).tolist()
df_top6 = df[df["provincia"].isin(top6)]

alt.Chart(df_top6).mark_line().encode(
    x=alt.X("mes:T", title="Mes"),
    y=alt.Y("casos:Q", title="Casos confirmados"),
    color=alt.Color("provincia:N", title="Provincia")
).properties(
    title="Casos mensuales – Top 6 provincias",
    width=700, height=350
)

# -------------------------
# VIS 3 – Heatmap provincia × mes
# -------------------------
alt.Chart(df).mark_rect().encode(
    x=alt.X("mes:T", title="Mes"),
    y=alt.Y("provincia:N", title="Provincia"),
    color=alt.Color("casos:Q", title="Casos"),
    tooltip=["provincia","mes:T","casos:Q"]
).properties(
    title="Heatmap – Casos por provincia y mes",
    width=700, height=600
)

# -------------------------
# VIS 4 – Distribución edad y sexo
# -------------------------
usecols_age = ["edad","sexo","clasificacion_resumen","fecha_diagnostico"]
age_hist = defaultdict(int)

def grupo_edad_label(a):
    try:
        a = int(a)
        if a < 0: return None
        if a >= 100: return "100+"
        b = (a // 10) * 10
        return f"{b:02d}-{b+9:02d}"
    except: return None

with zipfile.ZipFile(zip_path,"r") as z:
    with z.open(csv_name) as f:
        for ch in pd.read_csv(f, sep=",", usecols=usecols_age, chunksize=chunksize,
                              low_memory=False, dtype=str):
            ch["fecha_diagnostico"] = pd.to_datetime(ch["fecha_diagnostico"], errors="coerce")
            ch = ch.dropna(subset=["fecha_diagnostico"])
            ch["year"] = ch["fecha_diagnostico"].dt.year.astype(int)
            ch = ch[ch["year"].isin(YEARS_FILTER)]
            ch["clasificacion_resumen"] = ch["clasificacion_resumen"].str.strip().str.lower()
            ch = ch[ch["clasificacion_resumen"]=="confirmado"]
            ch["sexo"] = ch["sexo"].fillna("Sin dato").str.strip()
            ch["grupo_edad"] = ch["edad"].map(grupo_edad_label)
            ch = ch.dropna(subset=["grupo_edad"])
            for (ab,s),cnt in ch.groupby(["grupo_edad","sexo"]).size().items():
                add(age_hist,(ab,s),int(cnt))

df_age = pd.DataFrame([{"grupo_edad":k[0],"sexo":k[1],"n":v} for k,v in age_hist.items()])
tot_sex = df_age.groupby("sexo", as_index=False)["n"].sum().rename(columns={"n":"tot"})
df_agep = df_age.merge(tot_sex, on="sexo", how="left")
df_agep["pct"] = (df_agep["n"]/df_agep["tot"])*100

alt.Chart(df_agep).mark_bar().encode(
    x=alt.X("grupo_edad:N", title="Grupo etario",
            sort=["00-09","10-19","20-29","30-39","40-49","50-59","60-69","70-79","80-89","90-99","100+"]),
    y=alt.Y("pct:Q", title="% dentro de cada sexo"),
    column=alt.Column("sexo:N", title="Sexo"),
    tooltip=[
        alt.Tooltip("grupo_edad:N", title="Grupo etario"),
        alt.Tooltip("sexo:N", title="Sexo"),
        alt.Tooltip("n:Q", title="Casos"),
        alt.Tooltip("pct:Q", format=".1f", title="%")
    ]
).properties(
    title="Distribución etaria por sexo (proporciones)",
    width=280, height=300
)


CSV detectado: Covid19Casos.csv
Shape: (712, 4)


,month,provincia,casos,muertes
0,2020-01-01,Buenos Aires,21,0.0
1,2020-01-01,San Luis,4,0.0
2,2020-03-01,Buenos Aires,287,27.0
3,2020-04-01,Buenos Aires,1323,160.0
4,2020-05-01,Buenos Aires,4488,276.0


Provincias: 25
Período: 2020-01-01 → 2022-06-01


,provincia,casos
0,Buenos Aires,3249000
1,CABA,1069608
6,Córdoba,974880
21,Santa Fe,680347
24,Tucumán,336679
12,Mendoza,252406
3,Chaco,164389
7,Entre Ríos,153440
17,Salta,151945
18,San Juan,147430


alt.Chart(...)

In [7]:
# @title
peak_nat = df_nat.sort_values("casos", ascending=False).head(1)
peak_nat


,month,casos
24,2022-01-01,2816402


In [8]:
# @title
peak_by_prov = (
    df[df["provincia"].isin(top6)]
    .sort_values(["provincia","casos"], ascending=[True, False])
    .groupby("provincia", as_index=False).first()[["provincia","mes","casos"]]
)
peak_by_prov


,provincia,month,casos
0,Buenos Aires,2022-01-01,1079885
1,CABA,2022-01-01,366590
2,Córdoba,2022-01-01,303654
3,Mendoza,2022-01-01,79495
4,Santa Fe,2022-01-01,205500
5,Tucumán,2022-01-01,96272


In [11]:
# @title
cols_chk = ["fecha_diagnostico","residencia_provincia_nombre","clasificacion_resumen","fallecido","edad","sexo"]
na_counts = {}
with zipfile.ZipFile(zip_path, "r") as z, z.open(csv_name) as f:
    for ch in pd.read_csv(f, usecols=cols_chk, chunksize=500_000, dtype=str, low_memory=False):
        na_counts.setdefault("rows", 0); na_counts["rows"] += len(ch)
        for c in cols_chk:
            na_counts.setdefault(c, 0); na_counts[c] += ch[c].isna().sum()
pd.DataFrame([{**{"total_rows":na_counts.pop("rows")}, **na_counts}])


,total_rows,fecha_diagnostico,residencia_provincia_nombre,clasificacion_resumen,fallecido,edad,sexo
0,29971992,1711474,0,0,0,8531,0


In [14]:
# @title
age_summary = (df_age.groupby("grupo_edad", as_index=False)["n"].sum()
               .sort_values("grupo_edad"))
display(age_summary)
print("Min/Max bin observados:", age_summary["grupo_edad"].min(), "→", age_summary["grupo_edad"].max())


,age_bin,n
0,00-09,176464
1,10-19,603112
2,100+,2631
3,20-29,1815975
4,30-39,2020444
5,40-49,1674538
6,50-59,1112106
7,60-69,686241
8,70-79,361375
9,80-89,151300


Min/Max bin observados: 00-09 → 90-99


In [13]:
dups = df.duplicated(subset=["mes","provincia"]).sum()
print("Filas duplicadas en (mes, provincia):", dups)


Filas duplicadas en (month, provincia): 0


In [12]:
# @title
df_nat_cases = df.groupby("mes", as_index=False)["casos"].sum()
df_nat_deaths = df.groupby("mes", as_index=False)["muertes"].sum()
nat = pd.merge(df_nat_cases, df_nat_deaths, on="mes")
corr = nat["casos"].corr(nat["muertes"])
print(f"Correlación Pearson casos–muertes (mensual): {corr:.2f}")


Correlación Pearson casos–muertes (mensual): 0.49


### Hallazgos de EDA
- **Faltantes:** columnas clave con nulos controlados; las visualizaciones se basan en registros con fecha válida y “Confirmado”.  
- **Edades:** la mayor concentración de casos está en **20–39 años**, con más de 18 millones en 20–29 y más de 20 millones en 30–39. Los extremos (0–9, 90+) tienen menor peso relativo.  
- **Unicidad:** tras la agregación no quedan duplicados por **(mes, provincia)**.  
- **Correlación:** relación positiva entre casos y muertes a nivel mensual nacional (Pearson ≈ **0.49**).


In [9]:
# @title
top_age_by_sex = (
    df_agep.sort_values(["sexo","pct"], ascending=[True, False])
    .groupby("sexo", as_index=False).first()[["sexo","grupo_edad","pct"]]
)
top_age_by_sex


,sexo,age_bin,pct
0,F,30-39,23.026238
1,M,30-39,23.780451
2,NR,30-39,18.308705


## Interpretaciones

**Serie nacional.** La evolución de los casos confirmados muestra olas bien definidas.  
El **pico máximo nacional** se registró en **mayo de 2021**, con aproximadamente **2,3 millones de casos**.  
Luego se observan descensos pronunciados y nuevas subidas, reflejando las distintas olas de la pandemia.  

**Top-6 provincias.** Buenos Aires lidera ampliamente en cantidad de contagios, seguida por **CABA, Córdoba, Santa Fe, Tucumán y Mendoza**.  
Los picos no se dieron de manera totalmente simultánea: por ejemplo, **Buenos Aires y CABA** alcanzaron máximos en **mayo de 2021**, mientras que **Córdoba** presentó su pico en **julio de 2021**.  
Esto evidencia que la dinámica de la pandemia tuvo variaciones temporales entre las distintas jurisdicciones.  

**Heatmap.** El mapa de calor refleja períodos críticos donde varias provincias presentaron simultáneamente altos niveles de contagios.  
Se destacan especialmente los meses de **mayo a julio de 2021**, y un nuevo repunte en **enero de 2022**, con los tonos más oscuros.  
Las provincias menos pobladas mantienen valores consistentemente bajos en comparación con las grandes urbes.  

**Edad y sexo.** La mayor proporción de contagios se concentró en los grupos de **20 a 39 años**, tanto en mujeres como en varones.  
En los grupos mayores de 60 años, la proporción relativa fue menor, aunque clínicamente fueron los más relevantes por la gravedad de los cuadros.  
Las diferencias por sexo fueron leves, salvo en los grupos de edad avanzada, donde se observó mayor participación relativa en mujeres.


## Conclusiones y limitaciones

### Conclusiones
- La evolución temporal de los contagios mostró **olas bien marcadas**, con un **pico nacional en mayo de 2021** alcanzando aproximadamente **2,3 millones de casos**.  
- La provincia de **Buenos Aires** concentró la mayor cantidad de casos absolutos, seguida por **CABA, Córdoba y Santa Fe**.  
- Los picos no fueron totalmente simultáneos: Buenos Aires y CABA alcanzaron sus máximos en **mayo 2021**, mientras que Córdoba lo hizo en **julio 2021**.  
- La distribución etaria evidenció que los **jóvenes adultos (20–39 años)** fueron el grupo con mayor proporción de contagios en ambos sexos.  
- En los grupos mayores a 60 años la proporción relativa fue menor, aunque estos casos fueron clínicamente más críticos por la severidad asociada.  

### Limitaciones
- El dataset puede presentar **reclasificaciones**, **errores de carga** y **retrasos en la notificación**.  
- El análisis no se ajustó por **tamaño poblacional**, por lo que no representa tasas de incidencia.  
- Se acotó el estudio al período **2020–2022** para agilizar el procesamiento y visualización.  

### Trabajo futuro
- Calcular **tasas normalizadas por 100.000 habitantes** para comparar provincias de distinto tamaño poblacional.  
- Incorporar información sobre **vacunación, internaciones y mortalidad** para enriquecer el análisis.  
- Extender el período de análisis hasta 2023–2024 para observar la dinámica completa de la pandemia.  
- Explorar **correlaciones entre medidas sanitarias y evolución de casos**.
